In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import os
import shutil
import time

In [4]:
img_folder = './images'

if not os.path.isdir(img_folder) : 
    os.mkdir(img_folder)

In [15]:
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
# options.add_argument("headless")
driver = webdriver.Chrome(options=options)
index = 1
for page in range(1, 20):
    time.sleep(5)
    url = 'https://www.musinsa.com/mz/brandsnap?_m=&gender=&mod=&bid=&p=' + str(page)
    driver.get(url=url)
    driver.implicitly_wait(2)
    ul_element = driver.find_element(By.CSS_SELECTOR, '#wrapper > div.bottom-column.column.clearfix > div.main-content-wrapper > div.section > div > div > div.list-box.box > ul')
    
    li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
    # 각 <li> 태그를 순회
    for i, li in enumerate(li_elements):
        # <a> 태그 찾기
        a_element = li.find_element(By.TAG_NAME, 'a')

        # <a> 태그의 링크를 가져온 후 클릭
        a_link = a_element.get_attribute('href')
        driver.get(a_link)
        # 새 페이지에서 이미지 찾기
        # (이미지 태그의 정확한 위치에 따라 셀렉터 수정 필요)
        img_element = driver.find_element(By.CSS_SELECTOR, '#wrapper > div.bottom-column.column.clearfix > div.main-content-wrapper > div.content-wrapper.article.wrapper > div:nth-child(1) > div.staff-info-box.info-box.box.clearfix > div.snapImg > div.swiper-container.swiper-container-initialized.swiper-container-horizontal > div.swiper-wrapper > div > img')
        img_url = img_element.get_attribute('src')

        # 이미지 다운로드 및 저장
        response = requests.get(img_url, stream=True)
        with open(f'{img_folder}/image_{index}.jpg', 'wb') as file:
            shutil.copyfileobj(response.raw, file)
        del response

        index += 1

        # 원래 페이지로 돌아감
        driver.back()


# 웹드라이버 종료
driver.quit()



In [24]:
import os
import shutil

# 폴더 경로 설정
source_folder = './images'
target_folder = './labeled_img'

# labeled_img 폴더가 없으면 생성
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# images 폴더에서 .txt 확장자 파일 검색
for file_name in os.listdir(source_folder):
    if file_name == 'classes.txt': continue
    try:
        if file_name.endswith('.txt'):
            base_name = os.path.splitext(file_name)[0]

            # .txt 파일과 동일한 이름의 .jpg 파일
            jpg_file = f'{base_name}.jpg'

            # .txt 파일과 .jpg 파일 이동
            shutil.move(os.path.join(source_folder, file_name), os.path.join(target_folder, file_name))
            shutil.move(os.path.join(source_folder, jpg_file), os.path.join(target_folder, jpg_file))
    except:
        continue

In [30]:
import random

split_ratio = 0.1

img_file_name_list = list(map(lambda x : x.split('.')[0], os.listdir('./labeled_img')))

split_num = int(len(img_file_name_list) * split_ratio)
random.shuffle(img_file_name_list)

move_file_name_list = img_file_name_list[:split_num]

print('=======================================')
print('train -> valid 10% / start moving...')

for file_name in move_file_name_list:
    try:

        shutil.move(os.path.join('./labeled_img',file_name) + '.jpg',
                    os.path.join('./valid',file_name) + '.jpg')
        
        shutil.move(os.path.join('./labeled_img',file_name) + '.txt',
                    os.path.join('./valid',file_name) + '.txt')
    except:
        continue

print('=======================================')
print('train nums : ', len(os.listdir('./labeled_img')),'\ntrain nums : ', len(os.listdir('./labeled_img')))
print('valid nums : ', len(os.listdir('./valid')),'\nvalid nums : ', len(os.listdir('./valid')))
print('=======================================')

train -> valid 10% / start moving...
train nums :  553 
train nums :  553
valid nums :  180 
valid nums :  180


In [31]:
import os
import shutil

# 기본 폴더 경로
train_dir = './train'
images_dir = os.path.join(train_dir, 'images')
labels_dir = os.path.join(train_dir, 'labels')

# 폴더가 없으면 생성
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)

# train 폴더 내의 모든 파일 확인
for file_name in os.listdir(train_dir):
    # 파일의 전체 경로
    file_path = os.path.join(train_dir, file_name)

    # 이미지 파일인 경우
    if file_name.endswith('.jpg'):
        # train/images로 이동
        shutil.move(file_path, os.path.join(images_dir, file_name))

    # 레이블 파일인 경우
    elif file_name.endswith('.txt'):
        # train/labels로 이동
        shutil.move(file_path, os.path.join(labels_dir, file_name))


In [32]:
import os
import shutil

# 기본 폴더 경로
train_dir = './valid'
images_dir = os.path.join(train_dir, 'images')
labels_dir = os.path.join(train_dir, 'labels')

# 폴더가 없으면 생성
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)

# train 폴더 내의 모든 파일 확인
for file_name in os.listdir(train_dir):
    # 파일의 전체 경로
    file_path = os.path.join(train_dir, file_name)

    # 이미지 파일인 경우
    if file_name.endswith('.jpg'):
        # train/images로 이동
        shutil.move(file_path, os.path.join(images_dir, file_name))

    # 레이블 파일인 경우
    elif file_name.endswith('.txt'):
        # train/labels로 이동
        shutil.move(file_path, os.path.join(labels_dir, file_name))

In [43]:
import yaml

with open('./data.yaml','r') as f:
    data = yaml.safe_load(f)

print(data)

data['train'] = './train/images/' # train 경로
data['val'] = './valid/images/' # validation 경로

data['nc'] = 32 # class 개수
data['names'] = [
    "Beanie",
    "Buckethat",
    "Cap",
    "Hat",
    "Glasses",
    "Coat",
    "Bubble jacket",
    "Outer",
    "Jacket",
    "Shorts",
    "Skirt",
    "Croptop",
    "T-shirts",
    "Sweatshirt",
    "Shirt",
    "Vest",
    "Hoodie",
    "Top",
    "Jeans",
    "Training pants",
    "Pants",
    "Shoes",
    "Boots",
    "Sandal",
    "Muffler",
    "Tie",
    "Mask",
    "Headphone",
    "Accessories",
    "Handbag",
    "Backpack",
    "Bag"
] # class 이름

with open('./data.yaml','w') as f:
    yaml.dump(data, f)

print(data)

{'train': './train/images', 'val': './valid/images', 'nc': 2, 'names': ['mask', 'no-mask']}
{'train': './train/images/', 'val': './valid/images/', 'nc': 32, 'names': ['Beanie', 'Buckethat', 'Cap', 'Hat', 'Glasses', 'Coat', 'Bubble jacket', 'Outer', 'Jacket', 'Shorts', 'Skirt', 'Croptop', 'T-shirts', 'Sweatshirt', 'Shirt', 'Vest', 'Hoodie', 'Top', 'Jeans', 'Training pants', 'Pants', 'Shoes', 'Boots', 'Sandal', 'Muffler', 'Tie', 'Mask', 'Headphone', 'Accessories', 'Handbag', 'Backpack', 'Bag']}
